# Detección de barras en galaxias - Modelo "Baseline"
## Proyecto integrador MNA

### Integrantes
- Jonathan Jesús Marmolejo Hernández - A01795195
- Isaid Posadas Oropeza - A01795015
- Luis Daniel Ortega Muñoz - A01795197

In [ ]:
# Uncomment this if running in Google Colab. It will install the bargal package from GitHub.
# !pip install git+https://github.com/ludanortmun/itesm-mna-barred-galaxies.git

## Importando librerías

In [ ]:
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from PIL import Image

from bargal.dataset.load import load_dataset

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

## Preparando el conjunto de datos

El primer paso consiste en cargar nuestro conjunto de datos y dividirlo en conjunto de entrenamiento, validación y prueba.

In [ ]:
dataset_path = '../data/dataset.csv'

df = load_dataset(dataset_path)

train_df, val_test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['Bars'])
valid_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42, stratify=val_test_df['Bars'])

print(f'Train size: {len(train_df)}')
print(f'Validation size: {len(valid_df)}')
print(f'Test size: {len(test_df)}')

Ahora creamos nuestra clase para representar el conjunto de datos, heredando de Dataset de PyTorch. El dataset a utilizar para nuestra red neuronal consiste en imágenes de galaxias y su respectiva etiqueta, que indica si la galaxia tiene o no una barra.

La etiqueta es un tensor de tamaño 1, donde 1 indica que la galaxia tiene una barra y 0 indica que no. Sin embargo, el conjunto de datos original no tiene etiquetas binarias, sino múltiples categorías indicando el tipo de barra que tiene la galaxia. Por lo tanto, convertimos cualquier etiqueta que represente la presencia de una barra (independientemente de sus características) a 1 y cualquier etiqueta que represente la ausencia de una barra a 0. También debemos filtrar elementos con la etiqueta -0.5, la cual simboliza que la categoria de la galaxia es desconocida. Estas transformaciones fueron exploradas previamente en el entregable [Avance1.Equipo22.ipynb](https://github.com/ludanortmun/itesm-mna-barred-galaxies/tree/main/notebooks/Avance1.Equipo22.ipynb), donde se creó la columna `has_bar` derivada de `Bars` y se filtraron los elementos sin clasificación.

En cuanto a la carga de imágenes, este conjunto de datos consiste en las imágenes de galaxias previamente procesadas en formato PNG. El preprocesamiento consiste, principalmente, en la sustracción de las bandas G y R para enfatizar las estructuras de barras. Las imágenes resultantes tienen dimensiones de 400x400 píxeles y están en escala de grises. Estas imágenes son cargadas utilizando la librería PIL y convertidas a tensores.

El script de preprocesamiento puede ser consultado en este enlace: [bargal/commands/preprocess.py](https://github.com/ludanortmun/itesm-mna-barred-galaxies/blob/297f69b278ea6bc5099ef23a0d539602995bc55e/bargal/commands/preprocess.py)

El conjunto de imágenes pre procesadas puede descargarse con el siguiente enlace: [dataset.processed.GRLogDiff](https://tecmx-my.sharepoint.com/:u:/g/personal/a01795197_tec_mx/EexaLnqaLLdCt1JNxLib8VYBeOHJo95vuOr-Pfxv-55Iww?e=0gfeuq)


In [ ]:
class GalaxiesDataset(Dataset):
    def __init__(self, galaxies_df: pd.DataFrame, img_dir: str):
        self.filtered_df = galaxies_df[galaxies_df['Bars'] >= 0].reset_index(drop=True)
        self.img_dir = img_dir

    def __len__(self):
        return len(self.filtered_df)

    def __getitem__(self, idx):
        row = self.filtered_df.iloc[idx]
        label = 1.0 if row['Bars'] != 0 else 0.0
        img_path = f"{self.img_dir}/{row['name']}.png"

        # Load image and convert to tensor
        with Image.open(img_path) as img:
            image = transforms.ToTensor()(img)

        return image.to(device), torch.tensor([label], dtype=torch.float32).to(device)

In [ ]:
n = 8
processed_images_path = '../data/processed'

train_data = GalaxiesDataset(train_df, processed_images_path)
train_loader = DataLoader(train_data, batch_size=n, shuffle=True)
train_N = len(train_loader.dataset)

valid_data = GalaxiesDataset(valid_df, processed_images_path)
valid_loader = DataLoader(valid_data, batch_size=n)
valid_N = len(valid_loader.dataset)

## Definiendo el modelo

Debido a que estamos trabajando con imágenes, utilizaremos una red neuronal convolucional (CNN) como modelo base. La arquitectura de la red es la siguiente:

In [ ]:
# TODO: define the model architecture

model = nn.Sequential(
    # First conv layer
    nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1),  # 8 x 400 x 400
    nn.BatchNorm2d(8),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2), # 8 x 200 x 200

    ## Flattening
    nn.Flatten(),
    nn.Linear(8*200*200, 512),
    nn.Dropout(.3),
    nn.ReLU(),
    nn.Linear(512, 1)
)

In [ ]:
model = torch.compile(model.to(device))
model

### Función de pérdida y optimizador

Debido a que estamos trabajando con un problema de clasificación binaria, utilizaremos la función de pérdida `BCEWithLogitsLoss`. El optimizador a utilizar es Adam.

In [ ]:
loss_function = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters())
my_model = model.to(device)

## Entrenamiento

### Definiendo la función de entrenamiento

In [ ]:
def get_batch_accuracy(output, y, N):
    zero_tensor = torch.tensor([0]).to(device)
    pred = torch.gt(output, zero_tensor)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

In [ ]:
def train() -> tuple[float, float]:
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)

    return loss, accuracy

In [ ]:
def validate() -> tuple[float, float]:
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)

    return loss, accuracy

### Ejecución del entrenamiento

In [ ]:
epochs = 20
prev_loss = float('inf')
min_delta = 0.001

for epoch in range(epochs):
    train_loss, train_acc = train()
    valid_loss, valid_acc = validate()
    print(f'Epoch {epoch}: Train Loss={train_loss:.4f}, Acc={train_acc:.4f} | Valid Loss={valid_loss:.4f}, Acc={valid_acc:.4f}')

    if abs(prev_loss - valid_loss) < min_delta:
        print('Stopping early')

### Guardando el modelo

In [ ]:
model_path = '../models/model.pth'
torch.save(model.state_dict(), model_path)

## Evaluación del modelo

Una vez entrenada la red neuronal, procedemos a evaluar su desempeño en el conjunto de prueba.

In [ ]:
# TODO: Using the train dataset, compute classification report, confusion matrix, etc.